In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
%%time
df_train = pd.read_csv('spam_train_bag_of_words.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('spam_test_bag_of_words.csv', index_col='Unnamed: 0')

Wall time: 43.5 s


In [3]:
df_test.head()

,Label,go,jurong,point,crazi,avail,bugi,n,great,world,...,guy_bitch,bitch_act,act_like,like_interest,interest_buy,buy_someth,els_next,week_gave,gave_us,us_free
text1,ham,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
text2,spam,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
text3,ham,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
text4,ham,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
text5,spam,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_test.shape

(1671, 29155)

In [5]:
def split_features_label(data_df, label):
    return data_df.drop(columns=[label]), data_df[label]

def scale_split(df_train, df_test, label):
    X_train, y_train = split_features_label(df_train, label)
    X_test, y_test = split_features_label(df_test, label)

    scaler = MinMaxScaler().fit(X_train)
    X_train_sc = scaler.transform(X_train)
    X_test_sc = scaler.transform(X_test)
    X_train_sc = pd.DataFrame(X_train_sc, columns=X_train.columns, index=X_train.index)
    X_test_sc = pd.DataFrame(X_test_sc, columns=X_test.columns, index=X_test.index)
    return X_train_sc, X_test_sc, y_train, y_test

def build_encoder_films(encoding_dim, X_tr, X_te):
    ncol = X_tr.shape[1]
    input_dim = Input(shape = (ncol, ))
    encoded1 = Dense(5000, activation = 'relu')(input_dim)
    encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
    decoded1 = Dense(5000, activation = 'relu')(encoded2)
    output = Dense(ncol, activation = 'sigmoid')(decoded1)
    autoencoder = Model(inputs = input_dim, outputs = output)
    autoencoder.compile(optimizer = 'adadelta', loss = 'mean_squared_error')
    autoencoder.summary()
    autoencoder.fit(X_tr, X_tr, nb_epoch = 1, batch_size = 100, shuffle = False, validation_data = (X_te, X_te))
    encoder = Model(inputs = input_dim, outputs = encoded2)
    return encoder

def encode_train_test_films_to_csv(encoder, X_train_sc, X_test_sc, y_train, y_test, encod_dim):
    X_train_encoded = pd.DataFrame(encoder.predict(X_train_sc)).add_prefix('feature_')
    X_train_encoded.index = X_train_sc.index
    X_train_fin = pd.concat([X_train_encoded, y_train], axis=1)
    X_train_fin.to_csv("spam_train_ae_compressed_to_" + str(encod_dim) + "_features.csv", index_label='id_of_the_text')

    X_test_encoded = pd.DataFrame(encoder.predict(X_test_sc)).add_prefix('feature_')
    X_test_encoded.index = X_test_sc.index
    X_test_fin = pd.concat([X_test_encoded, y_test], axis=1)
    X_test_fin.to_csv("spam_test_ae_compressed_to_" + str(encod_dim) + "_features.csv", index_label='id_of_the_text')
    return 'hoi'

In [6]:
label = 'Label'
encod_dim = 500

In [7]:
X_train_sc, X_test_sc, y_train, y_test = scale_split(df_train, df_test, label)
X_train_ae, X_test_ae = train_test_split(X_train_sc, train_size = 0.85, random_state=42)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
%%time
encoder = build_encoder_films(encod_dim, X_train_ae, X_test_ae)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 29154)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5000)              145775000 
_________________________________________________________________
dense_2 (Dense)              (None, 500)               2500500   
_________________________________________________________________
dense_3 (Dense)              (None, 5000)              2505000   
_________________________________________________________________
dense_4 (Dense)              (None, 29154)             145799154 
Total params: 296,579,654
Trainable params: 296,579,654
Non-trainable params: 0
_________________________________________________________________


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3315 samples, validate on 586 samples
Epoch 1/1
2300/3315 [===================>..........] - ETA: 1:05 - loss: 0.2499

In [ ]:
%%time
encode_train_test_films_to_csv(encoder, X_train_sc, X_test_sc, y_train, y_test, encod_dim)